In [1]:
import os
try:
  if not os.path.exists('/content/sample_submission.csv') and not os.path.exists('/content/train.csv.zip') and not os.path.exists('/content/test.csv'):
    ! mkdir ~/.kaggle
    ! mv kaggle.json ~/.kaggle/
    ! chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c google-quest-challenge
except:
  pass

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory
  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 42.5MB/s]
  0% 0.00/954k [00:00<?, ?B/s]
100% 954k/954k [00:00<00:00, 62.1MB/s]
  0% 0.00/4.50M [00:00<?, ?B/s]
100% 4.50M/4.50M [00:00<00:00, 148MB/s]


In [2]:
def data_directory():
  try:
    if not os.path.exists('/content/Google_Quest/train_data')and not os.path.exists('/content/Google_Quest/test_data') and not os.path.exists('/content/Google_Quest/sample_data') :
      !unzip train.csv -d /content/Google_Quest/train_data
      !mv test.csv /content/Google_Quest/test_data
      ! mkdir Google_Quest
      ! mkdir /content/Google_Quest/train_data
      ! mkdir /content/Google_Quest/test_data
      ! mkdir /content/Google_Quest/sample_data
      !unzip train.csv -d /content/Google_Quest/train_data
      !mv test.csv /content/Google_Quest/test_data
      !mv sample_submission.csv /content/Google_Quest/sample_data
  except:
    pass

  return {
      'train_data_path':'/content/Google_Quest/train_data/train.csv',
      'test_data_path':'/content/Google_Quest/test_data/test.csv',
      'sample_data_path':'/content/Google_Quest/sample_data/sample_submission.csv'
    }

In [18]:
import torch.nn as nn
import transformers
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AdamW,get_linear_schedule_with_warmup
from tqdm import  tqdm 
import numpy as np

In [4]:
def get_device():
  if torch.cuda.is_available():
    return torch.device('cuda:0')
  else:
    return torch.device('cpu')


In [5]:
class Bertmodel(nn.Module):
  def __init__(self):
    super(Bertmodel,self).__init__()
    self.bert=transformers.BertModel.from_pretrained('bert-base-uncased')
    self.bert_drop=nn.Dropout(0.3)
    self.out=nn.Linear(768,30)
  
  def forward(self, ids,mask,token_type_ids):
    _,output2=self.bert(ids,attention_mask=mask,token_type_ids=token_type_ids)
    bo=self.bert_drop(output2)
    return self.out(bo)


In [6]:


class datatraining:
  def __init__(self,qtitle,qbody,answer,tokenizer,max_len,targets):
    self.qtitle=qtitle
    self.qbody=qbody
    self.answer=answer
    self.tokenizer=tokenizer
    self.max_len=max_len
    self.targets=targets
  
  def __len__(self):
    return len(self.answer)
  
  def __getitem__(self,item):
    question_title=str(self.qtitle[item])
    question_body=str(self.qbody[item])
    answer=str(self.answer[item])
    inputs=self.tokenizer.encode_plus(
                                      question_title+' '+ question_body,
                                      answer,
                                      add_special_tokens =True,
                                      truncation=True,
                                      max_length=self.max_len)
    ids=inputs['input_ids']
    token_type_ids=inputs['token_type_ids']
    mask=inputs['attention_mask']

    padding_len=self.max_len-len(ids)
    ids.extend([0]*padding_len)
    token_type_ids.extend([0]*padding_len)
    mask.extend([0]*padding_len)

    return{
        'ids':torch.tensor(ids,dtype=torch.long),
        'token_type_ids':torch.tensor(token_type_ids,dtype=torch.long),
        'mask':torch.tensor(mask,dtype=torch.long),
        'targets':torch.tensor(self.targets[item,:],dtype=torch.float)
        
    }
  

In [7]:
def loss_fn(outputs,targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)

In [8]:
def train_loop_fn(data_loader,model,optimizer,device,scheduler=None):
    model.train()
    for bi,d in enumerate(data_loader):
      ids=d['ids']
      mask=d['mask']
      token_type_ids=d['token_type_ids']
      targets=d['targets']

      ids=ids.to(device,dtype=torch.long)
      mask=mask.to(device,dtype=torch.long)
      token_type_ids=token_type_ids.to(device,dtype=torch.long)
      targets=targets.to(device,dtype=torch.float)

      optimizer.zero_grad()
      outputs=model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids
      )
      loss=loss_fn(outputs,targets)
      loss.backward()
      optimizer.step()
      if scheduler is not None:
        scheduler.step()


In [15]:
def eval_loop_fn(data_loader,model,device):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    for bi,d in enumerate(data_loader):
      ids=d['ids']
      mask=d['mask']
      token_type_ids=d['token_type_ids']
      targets=d['targets']

      ids=ids.to(device,dtype=torch.long)
      mask=mask.to(device,dtype=torch.long)
      token_type_ids=token_type_ids.to(device,dtype=torch.long)
      targets=targets.to(device,dtype=torch.float)

      outputs=model(
          ids=ids,
          mask=mask,
          token_type_ids=token_type_ids
      )
      loss=loss_fn(outputs,targets)

      fin_targets.append(targets.cpu().detach().numpy())
      fin_outputs.append(outputs.cpu().detach().numpy())

      return np.vstack(fin_outputs),np.vstack(fin_targets)

In [20]:
def run():
  MAX_LEN=512
  TRAIN_BATCH_SIZE=4
  EPOCHS=5

  data_dic=data_directory()

  df=pd.read_csv(data_dic['train_data_path']).fillna('none')
  df_train,df_valid=train_test_split(df,test_size=0.1,random_state=42)
  df_train.reset_index(drop=True)
  df_valid.reset_index(drop=True)

  tokenizer=transformers.BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)

  sample=pd.read_csv(data_dic['sample_data_path'])
  target_cols=list(sample.drop('qa_id',axis=1).columns)
  train_target=df_train[target_cols].values
  valid_target=df_valid[target_cols].values

  train_dataset=datatraining(
      qtitle=df_train.question_title.values,
      qbody=df_train.question_body.values,
      answer=df_train.answer.values,
      targets=train_target,
      tokenizer=tokenizer,
      max_len=MAX_LEN
  )
  train_data_loader=torch.utils.data.DataLoader(
                                                  train_dataset,
                                                  shuffle=True,
                                                  batch_size=TRAIN_BATCH_SIZE
  )
  valid_dataset=datatraining(
      qtitle=df_valid.question_title.values,
      qbody=df_valid.question_body.values,
      answer=df_valid.answer.values,
      targets=valid_target,
      tokenizer=tokenizer,
      max_len=MAX_LEN
  )
  valid_data_loader=torch.utils.data.DataLoader(
      valid_dataset,
      shuffle=True,
      batch_size=TRAIN_BATCH_SIZE
  )
  device=get_device()
  lr=2e-5
  num_train_steps=len(train_data_loader) * EPOCHS
  model=Bertmodel().to(device)
  optimizer=AdamW(model.parameters(),lr=lr)
  scheduler=get_linear_schedule_with_warmup(
                                            optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=num_train_steps
  )
  for epochs in tqdm(range(EPOCHS)):
    print(f'\nModel Training......  EPOCHS : {epochs}/{EPOCHS}' )
    train_loop_fn(train_data_loader,model,optimizer,device,scheduler)
    print('model validation.....')
    output,target=eval_loop_fn(valid_data_loader,model,device)
    print(output,target)  
  

In [21]:
if __name__ == '__main__':
  run()





  0%|          | 0/5 [00:00<?, ?it/s]


Model Training......  EPOCHS : 0/5





 20%|██        | 1/5 [10:40<42:41, 640.36s/it]

model validation.....
[[ 2.5453467   0.9968477  -1.7113839   1.0165261   0.4814835  -0.339422
   0.39409772  0.31171635 -1.7629433  -3.5713346   0.21047151 -1.6755514
  -2.6441495  -2.9351609  -2.5515666  -0.33480275 -1.059719   -1.939934
  -1.927981   -3.663052    1.3504947   1.609413    0.495539    2.5082881
   2.382195    1.4348254  -1.0609795  -1.9613885  -0.7847088   1.9549544 ]
 [ 1.8481889   0.43316537 -3.251773    0.59711665  1.7250323   1.6780798
   0.455279    0.14608094 -1.134621   -4.6586747  -0.45794886 -1.9082866
  -3.473966   -4.6336164  -3.7030396  -3.7233548  -0.34519592 -2.1099224
   1.4314215  -4.8424225   1.5334629   2.768669    0.57974595  3.2136996
   3.6812286   2.0618503  -0.43698877 -2.280026    1.2370201   2.4592812 ]
 [ 2.2515488   1.0949436  -2.4506118   1.5389166   1.6215472   1.5535314
   0.69984597  0.5425984  -0.71912616 -4.179669   -0.7479651   1.4067199
  -2.4766955  -3.7212913  -2.5123916  -2.4687772  -1.9181321  -2.4146583
  -0.12936415 -4.4417343   




 40%|████      | 2/5 [21:25<32:05, 641.74s/it]

...

model validation.....
[[ 2.1398427   0.46908122 -3.6656919   0.8771308   3.43766     2.531782
   0.21434398  0.15173289 -0.1952916  -4.751487   -2.6968555  -0.15547691
  -2.7521894  -3.7932167  -1.2017633  -1.7500885  -2.911949   -2.283942
   0.38326234 -4.8937907   1.7220454   2.3442032   0.7837496   2.8089607
   3.0405076   1.6452023  -2.7669392  -2.557886    1.3386285   2.213184  ]
 [ 2.932063    1.6300604  -1.8276925   2.2733572   1.0797917   0.18407759
   0.5227446   0.81344223 -2.5324807  -4.4416795  -0.7151533  -0.5879515
  -2.4482334  -3.9088376  -1.3810263  -0.38035905 -2.9781637  -2.9709752
  -2.4084704  -4.7105827   2.1287851   2.173222    0.65786475  2.6852942
   2.7425988   1.6041611  -2.536102   -2.92168    -1.4950875   2.2114718 ]
 [ 2.2935016   0.27036944 -4.169659    1.1388675   2.7676458   3.0042841
   0.3041064  -0.02851051 -0.66691697 -5.536743   -1.8688344   0.4442186
  -3.2006695  -4.296695   -3.4605417  -3.7188792  -2.139385   -2.3157828
   0.8743366  -6.056276  




 80%|████████  | 4/5 [42:58<10:44, 644.21s/it]

model validation.....
[[ 1.7806516   0.00914778 -4.1631975   1.1615933   0.9687561   2.1564415
   0.15319748 -0.24963978 -2.9433763  -4.914256    0.05132872 -2.8368351
  -4.647195   -5.1970677  -5.1028047  -5.0934987   0.8738355  -1.8064357
   0.699697   -6.455742    0.87717265  2.7289824   0.64513177  3.4996982
   3.9083765   2.0251029   0.7454324  -1.8468939   1.0726928   2.2512715 ]
 [ 2.2896445   0.15189219 -4.790868    0.52144665  1.5604037   2.175084
   0.16853315 -0.1200224  -0.44284758 -5.6845264  -0.12878412 -0.6513606
  -4.8807554  -5.1432395  -5.661449   -4.6186833   2.4106507  -0.7554921
  -1.9034458  -6.7902813   1.3060664   3.0666366   0.8225225   3.6725805
   4.0283895   1.9504961   2.0462098  -1.4518964  -0.9134653   2.6346178 ]
 [ 1.8674469  -0.1555829  -4.466754    0.48760906  1.1615795   1.6458652
   0.19482975 -0.1395724   0.5206426  -5.1324472   0.42453516  0.18905759
  -3.765227   -4.5511775  -5.775784   -3.8656573   1.73243    -0.86881334
  -1.2486631  -6.1716194




100%|██████████| 5/5 [53:45<00:00, 645.05s/it]

model validation.....
[[ 2.6962857   1.4386197  -4.703872    1.2789372   1.8645719   2.378204
   0.49391282 -0.05181449 -3.118688   -5.326174   -1.0505184  -3.950532
  -5.059736   -5.7333755  -4.7307444  -4.3775115   1.7630562  -1.43426
  -2.4893315  -6.696542    2.1083755   2.9325764   0.8394967   3.5318918
   4.012981    2.195619    1.0258366  -1.6997181  -0.9059772   2.5072641 ]
 [ 1.6228034  -0.21385801 -5.0574517   1.4092472   1.1757321   2.6967807
   0.09723268 -0.4657743  -2.9826894  -5.118979   -0.34807932 -2.5480778
  -5.1164556  -5.554128   -5.529633   -5.12297     1.1937456  -1.6940969
  -0.13563447 -6.7876987   0.6083834   3.1701353   0.7678815   3.6547554
   4.147084    2.0664625   0.9332692  -1.6979102   1.1524715   2.3019073 ]
 [ 2.2472713   0.6971499  -4.734753    1.0972497   1.4087214   2.022243
   0.33730707 -0.16674817 -3.2409048  -4.760838   -0.4454692  -4.2394147
  -4.7958775  -5.4068513  -4.9257946  -3.948426    2.0023785  -1.2048788
  -1.9955424  -6.3090153   1.3